In [111]:

!pip install pandas sqlalchemy selenium bs4 requests webdriver-manager

In [112]:

import pandas as pd
import requests
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from selenium import webdriver
import re 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager 
import time



In [113]:
## extracting data from coingecko API
url = "https://api.coingecko.com/api/v3/coins/markets"

params = {"vs_currency" :"usd",
          "order" : "market_cap_desc",
          "per_page" : 20,
          "page" : 1,
          "sparkline": False}
          
response = requests.get(url, params = params)


crypto_list = response.json()

crypto_df = pd.DataFrame(crypto_list)[["name", "symbol", "current_price", "market_cap", "ath_change_percentage"]]
crypto_df = crypto_df.rename(columns = {"ath_change_percentage": "price_change_percentage"})

crypto_df["data_source"] = "CoinGecko"
crypto_df

,name,symbol,current_price,market_cap,price_change_percentage,data_source
0,Bitcoin,btc,87362.000000,1743414203382,-30.69877,CoinGecko
1,Ethereum,eth,2827.730000,341222657953,-42.87296,CoinGecko
2,Tether,usdt,1.000000,184604671069,-24.41192,CoinGecko
3,XRP,xrp,2.040000,122916260474,-44.12630,CoinGecko
4,BNB,bnb,842.900000,116092474685,-38.44172,CoinGecko
5,USDC,usdc,0.999801,77439220915,-14.74410,CoinGecko
6,Solana,sol,128.320000,71801008463,-56.24792,CoinGecko
7,TRON,trx,0.277728,26301325663,-35.58024,CoinGecko
8,Lido Staked Ether,steth,2825.920000,24484073520,-42.73257,CoinGecko
9,Dogecoin,doge,0.137049,20824537235,-81.27376,CoinGecko


In [114]:

## testing yahoofinance website response

url = "https://uk.finance.yahoo.com/markets/crypto/all/requests.get(url)"
headers = {"User-Agent" : "Mozilla/5.0"}
response = requests.get(url, headers =headers)
print(response.status_code)

200


In [115]:
## extracting data from yahoofinance
url = "https://uk.finance.yahoo.com/markets/crypto/all/"
headers = {"User-Agent" : "Mozilla/5.0"}
response = requests.get(url, headers= headers)
soup = BeautifulSoup(response.text, "html.parser")
soup

<!DOCTYPE html>

<html class="desktop neo-green dock-upscale" data-color-scheme="auto" data-color-theme-enabled="true" lang="en-GB" theme="auto">
<head>
<meta charset="utf-8"/>
<meta content="guce.yahoo.com" name="oath:guce:consent-host"/>
<link crossorigin="anonymous" href="//s.yimg.com" rel="preconnect"/><link href="//geo.yahoo.com" rel="preconnect"><link href="//query1.finance.yahoo.com" rel="preconnect"/><link href="//query2.finance.yahoo.com" rel="preconnect"/><link href="//consent.cmp.oath.com" rel="preconnect"/><link as="worker" href="/__rapidworker-1.2.js" rel="preload"/><link href="https://s.yimg.com/uc/finance/webcore/js/_staticFinProtobuf.adb9bf9ae7922cf2dccf.js" rel="preconnect"/><link href="https://cdn.jsdelivr.net/npm/protobufjs@7.1.2/dist/minimal/protobuf.min.js" rel="preconnect"/><script>window.finNeoPageStart = Date.now();</script><script>(function(){if(!window.YAHOO){window.YAHOO={}}; window.YAHOO.context={"consent":{"allowContentPersonalization":false,"allowCrossDevi

In [116]:
rows = soup.select("table tbody tr")

In [117]:
rows[0].find_all("td")[6].text.strip()

'1.743T'

In [118]:
finance = []
for row in rows [:11]:
        symbol = row.find_all("td")[0].text.strip().split(" ")[2].split("-")[0]
        name = row.find_all("td")[1].text.strip().split(" ") [0]
        price =row.find_all("td")[3].text.strip().split(" ")[0]
        market_cap = row.find_all("td")[6].text.strip()

        finance.append({"name" :name,
                        "symbol" : symbol,
                        "price" :price,
                        "market_cap": market_cap,
                        "data_source" : "YahooFinance"
                
        })

finance_df = pd.DataFrame(finance)





In [119]:
finance_df

,name,symbol,price,market_cap,data_source
0,Bitcoin,BTC,"87,344.30",1.743T,YahooFinance
1,Ethereum,ETH,"2,826.50",341.146B,YahooFinance
2,Tether,USDT,1.00,184.606B,YahooFinance
3,XRP,XRP,2.04,122.941B,YahooFinance
4,BNB,BNB,843.49,116.18B,YahooFinance
5,USDC,USDC,1.00,77.435B,YahooFinance
6,Solana,SOL,128.34,71.825B,YahooFinance
7,TRON,TRX,0.28,26.297B,YahooFinance
8,Lido,STETH,"2,824.40",24.482B,YahooFinance
9,Wrapped,WTRX,0.28,24.327B,YahooFinance


In [120]:
## Etracting data from coin market cap (a dynamic website)

In [121]:
# configuring the chromedriver
options = Options()
options.headless = True 
options.disable_gpu = True
options.no_sandbox = True


#initialize the webdriver
driver = webdriver.Chrome(service =Service(ChromeDriverManager().install()), options =options)

#defining the urlthat needed to be scraped
url = "https://coinmarketcap.com/"

#activating the driver to get the url content
time.sleep(5)
driver.get(url)

response = driver.page_source 
soup = BeautifulSoup(response, "html.parser")

soup



<html dir="ltr" lang="en"><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, shrink-to-fit=no" name="viewport"/><meta content="https://coinmarketcap.com/" property="og:url"/><link href="https://coinmarketcap.com/" rel="canonical"/><link href="https://coinmarketcap.com/ar/" hreflang="ar" rel="alternate"/><link href="https://coinmarketcap.com/bg/" hreflang="bg" rel="alternate"/><link href="https://coinmarketcap.com/cs/" hreflang="cs" rel="alternate"/><link href="https://coinmarketcap.com/da/" hreflang="da" rel="alternate"/><link href="https://coinmarketcap.com/de/" hreflang="de" rel="alternate"/><link href="https://coinmarketcap.com/el/" hreflang="el" rel="alternate"/><link href="https://coinmarketcap.com/" hreflang="en" rel="alternate"/><link href="https://coinmarketcap.com/es/" hreflang="es" rel="alternate"/><link href="https://coinmarketcap.com/fi/" hreflang="fi" rel=

In [122]:
rows = soup.select("tbody tr")

In [123]:
name =rows[1].find("p", class_= "sc-65e7f566-0 iPbTJf coin-item-name").text.strip()

name

'Bitcoin'

In [124]:
symbol = rows[0].find("p", class_= "sc-65e7f566-0 byYAWx coin-item-symbol").text.strip()
symbol

'CMC20'

In [125]:
price = rows[0].find("div", class_ = "sc-fa25c04c-0 eAphWs").text.strip()
price

'$183.50'

In [126]:
market_cap = rows[0].find("span", class_ = "sc-11478e5d-1 jfwGHx").text.strip()
market_cap

'$6,174,645'

In [127]:
market_list = []
for row in rows [:11]:
    name = row.find("p", class_= "sc-65e7f566-0 iPbTJf coin-item-name").text.strip()
    symbol = row.find("p", class_= "sc-65e7f566-0 byYAWx coin-item-symbol").text.strip()
    price = row.find("div", class_ = "sc-fa25c04c-0 eAphWs").text.strip()
    market_cap = row.find("span", class_ = "sc-11478e5d-1 jfwGHx").text.strip()
    market_list.append({"name" : name,
                       "symbol" : symbol,
                       "price" : price,
                       "market_cap" : market_cap,
                       "data_source" : "CoinMarketCap"}

    )


In [128]:
coin_df = pd.DataFrame(market_list)
coin_df

,name,symbol,price,market_cap,data_source
0,CoinMarketCap 20 Index DTF,CMC20,$183.50,"$6,174,645",CoinMarketCap
1,Bitcoin,BTC,"$87,384.94","$1,746,802,164,391",CoinMarketCap
2,Ethereum,ETH,"$2,828.31","$341,318,310,460",CoinMarketCap
3,Tether,USDT,$1.00,"$184,604,184,490",CoinMarketCap
4,XRP,XRP,$2.03,"$122,977,969,157",CoinMarketCap
5,BNB,BNB,$844.09,"$116,262,667,313",CoinMarketCap
6,USDC,USDC,$0.9999,"$77,446,136,891",CoinMarketCap
7,Solana,SOL,$128.35,"$71,832,109,265",CoinMarketCap
8,TRON,TRX,$0.2778,"$26,302,497,928",CoinMarketCap
9,Dogecoin,DOGE,$0.1371,"$20,845,978,078",CoinMarketCap


In [129]:
coin_df['price'] = coin_df['price'].str.replace('$', "").str.replace(",", "")
coin_df['market_cap'] = coin_df['market_cap'].str.replace('$', "").str.replace(",", "")
coin_df

,name,symbol,price,market_cap,data_source
0,CoinMarketCap 20 Index DTF,CMC20,183.50,6174645,CoinMarketCap
1,Bitcoin,BTC,87384.94,1746802164391,CoinMarketCap
2,Ethereum,ETH,2828.31,341318310460,CoinMarketCap
3,Tether,USDT,1.00,184604184490,CoinMarketCap
4,XRP,XRP,2.03,122977969157,CoinMarketCap
5,BNB,BNB,844.09,116262667313,CoinMarketCap
6,USDC,USDC,0.9999,77446136891,CoinMarketCap
7,Solana,SOL,128.35,71832109265,CoinMarketCap
8,TRON,TRX,0.2778,26302497928,CoinMarketCap
9,Dogecoin,DOGE,0.1371,20845978078,CoinMarketCap


In [130]:
crypto_df.head()

,name,symbol,current_price,market_cap,price_change_percentage,data_source
0,Bitcoin,btc,87362.00,1743414203382,-30.69877,CoinGecko
1,Ethereum,eth,2827.73,341222657953,-42.87296,CoinGecko
2,Tether,usdt,1.00,184604671069,-24.41192,CoinGecko
3,XRP,xrp,2.04,122916260474,-44.12630,CoinGecko
4,BNB,bnb,842.90,116092474685,-38.44172,CoinGecko


In [131]:
crypto_df = crypto_df.rename(columns = {'current_price': 'price'})[["name", "symbol", "price"]]
finance_df = finance_df [["name", "symbol", "price"]]
coin_df = coin_df [["name", "symbol", "price"]]

combined_df = pd.concat([crypto_df,
                         finance_df,
                         coin_df], ignore_index = True)


In [132]:
combined_df.head()

,name,symbol,price
0,Bitcoin,btc,87362.0
1,Ethereum,eth,2827.73
2,Tether,usdt,1.0
3,XRP,xrp,2.04
4,BNB,bnb,842.9
